<div style="text-align: center; margin-top: 30px; margin-bottom: 30px;">
    <img src="/Workspace/Users/nagham.omar@campus.technion.ac.il/VibeBnB/static/logo.png"
         alt="VibeBnB Logo"
         style="max-width: 350px;"/>
</div>




# Welcome to **VibeBnB**

**VibeBnB** helps you discover Airbnb listings with a similar *vibe* across European destinations.  
Instead of searching by location alone, we match places based on atmosphere, surroundings, and overall experience.

---

## What you can do here

1. **Filter listings** across Europe by **country**, **city**, **rating**, and **price**.
2. Choose a **reference listing** that matches your taste.
3. Select a **target country** and adjust your personal preferences.
4. Get **Top-K** recommended listings ranked by similarity and vibe.

---

> ⚠️ **Important – Setup Required**  
> Before starting the demo, please make sure to **run all notebook cells on the active cluster**.

🔗 **[View the VibeBnB Prototype](https://vibebnb-983293358114278.18.azure.databricksapps.com/)**






In [0]:
import json, time
import ipywidgets as widgets
from IPython.display import display, clear_output

from pyspark.sql import functions as F
from pyspark.storagelevel import StorageLevel
from pyspark.ml.feature import BucketedRandomProjectionLSHModel
from IPython.display import display, HTML

import time
import ipywidgets as widgets
import ipywidgets as W
from IPython.display import display as ipy_display, clear_output
from pyspark.sql import functions as F
from retrieve_rank import retrieve, order
EMBEDDING_PATH = "dbfs:/vibebnb/data/europe_countries_embedded"
DATA_PATH      = "dbfs:/vibebnb/data/europe_countries_scored.parquet"
LSH_MODEL_PATH = "dbfs:/vibebnb/models/lsh_global"

UI_COLS = [
    "property_id", "addr_cc", "listing_title", "room_type_text",
    "addr_name", "price_per_night", "ratings"
]
# Load once 
df_all = (
    spark.read.parquet(DATA_PATH)
    .dropDuplicates(["property_id"])
    .persist(StorageLevel.MEMORY_AND_DISK)
)

df_emb = (
    spark.read.parquet(EMBEDDING_PATH)
    .select("property_id", "addr_cc", "features_norm")
    .dropDuplicates(["property_id"])
    .persist(StorageLevel.MEMORY_AND_DISK)
)

lsh_model = BucketedRandomProjectionLSHModel.load(LSH_MODEL_PATH)

# Countries for dropdown
countries = [r["addr_cc"] for r in (
    df_all.select(F.upper(F.trim("addr_cc")).alias("addr_cc"))
         .where(F.col("addr_cc").isNotNull() & (F.col("addr_cc") != ""))
         .dropDuplicates(["addr_cc"])
         .orderBy("addr_cc")
         .collect()
)]

# --- Infer env columns (matches your infer_env_embedding_cols idea) ---
def infer_env_cols_from_columns(cols: list[str]) -> list[str]:
    env_norm = [c for c in cols if c.startswith("env_") and c.endswith("_norm")]
    if env_norm:
        return sorted(env_norm)
    env_raw = [c for c in cols if c.startswith("env_") and (not c.endswith("_max")) and (not c.endswith("_norm"))]
    return sorted(env_raw)

ENV_COLS = infer_env_cols_from_columns(df_all.columns)



import ipywidgets as widgets
from IPython.display import display as ipy_display, clear_output
import time

# User-facing column labels
DISPLAY_COL_RENAME = {
    "listing_title": "Listing",
    "addr_cc": "Country",
    "addr_name": "City",
    "room_type_text": "Room Type",
    "price_per_night": "Price (€ / night)",
    "ratings": "Rating",
    "final_score": "Match Score",
    "final_url": "Link"
}

def show_spark_df(
    df,
    n=50,
    title=None,
    cols=None,
    output_widget=None,
    rename_map=None
):
    if output_widget is None:
        raise ValueError("output_widget is required")

    rename_map = rename_map or {}

    if cols:
        df = df.select(*cols)

    rows = df.limit(n).collect()
    colnames = df.columns if not cols else cols

    def esc(x):
        s = "" if x is None else str(x)
        return s.replace("&","&amp;").replace("<","&lt;").replace(">","&gt;")

    header = "".join(
        f"<th style='text-align:left;padding:8px;border-bottom:1px solid rgba(255,255,255,.12);"
        f"color:#e9f1ff;background:#0b2340;position:sticky;top:0;'>"
        f"{esc(rename_map.get(c, c))}</th>"
        for c in colnames
    )

    body = ""
    for r in rows:
        body += "<tr>" + "".join(
            f"<td style='padding:8px;border-bottom:1px solid rgba(255,255,255,.08);color:#e9f1ff;'>"
            f"{esc(r[c])}</td>"
            for c in colnames
        ) + "</tr>"

    html = f"""
    <div style="overflow:auto; max-height:520px; border:1px solid rgba(255,255,255,.14);
                border-radius:14px; background:rgba(7,26,43,.65);
                box-shadow:0 10px 22px rgba(0,0,0,.35);">
      <table style="border-collapse:collapse; width:100%; font-family:system-ui; font-size:13px;">
        <thead><tr>{header}</tr></thead>
        <tbody>{body}</tbody>
      </table>
    </div>
    """

    with output_widget:
        clear_output(wait=True)
        if title:
            ipy_display(widgets.HTML(f"<h3 style='margin:8px 0;color:#e9f1ff'>{esc(title)}</h3>"))
        ipy_display(widgets.HTML(html))



## Step 1: Reference Property Selection

Apply filtering criteria to identify a reference property, which will be used as the query point for similarity-based retrieval.



In [0]:
w_country = widgets.Dropdown(options=countries, description="Country", layout=widgets.Layout(width="360px"))
w_city = widgets.Dropdown(options=[], description="City", layout=widgets.Layout(width="360px"))

w_rating = widgets.FloatRangeSlider(
    value=[0.0, 5.0], min=0.0, max=5.0, step=0.1,
    description="Rating", continuous_update=False,
    layout=widgets.Layout(width="520px")
)

PRICE_MAX_UI = 1300
w_price = widgets.IntRangeSlider(
    value=[0, PRICE_MAX_UI], min=0, max=PRICE_MAX_UI, step=10,
    description="Price", continuous_update=False,
    layout=widgets.Layout(width="520px")
)

btn_filter = widgets.Button(description="Filter & Show", button_style="primary")
out_filter = widgets.Output()

STATE = {"filtered_df": None, "filtered_rows": None, "final_topk": None}

def refresh_cities(cc):
    if not cc:
        return []
    rows = (
        df_all.where(F.col("addr_cc") == cc)
              .select(F.col("addr_name"))
              .where(F.col("addr_name").isNotNull())
              .dropDuplicates()
              .orderBy("addr_name")
              .collect()
    )
    return [r["addr_name"] for r in rows]

def on_country_change(change):
    cities = refresh_cities(change["new"])
    w_city.options = cities
    w_city.value = cities[0] if cities else None

w_country.observe(on_country_change, names="value")
on_country_change({"new": w_country.value})

def on_filter(_):
    with out_filter:
        clear_output(wait=True)
        df = (
            df_all
            .where(F.col("addr_cc") == w_country.value)
            .where(F.col("addr_name") == w_city.value)
            .where(F.col("ratings").between(*w_rating.value))
            .where(F.col("price_per_night").between(*w_price.value))
            .select(*UI_COLS)
            .orderBy(F.desc("ratings"), F.asc("price_per_night"))
        )

        rows = df.collect()
        STATE["filtered_df"] = df
        STATE["filtered_rows"] = rows

        show_spark_df(
            df,
            n=min(50, len(rows)),
            title="Filtered Listings",
            cols=[
                "listing_title",
                "addr_cc",
                "addr_name",
                "room_type_text",
                "price_per_night",
                "ratings"
            ],
            rename_map=DISPLAY_COL_RENAME,
            output_widget=out_filter
        )

btn_filter.on_click(on_filter)

ipy_display(widgets.VBox([
    widgets.HBox([w_country, w_city]),
    w_rating,
    w_price,
    btn_filter,
    out_filter
]))


## Step 2: Build Listing Choices and Set the Target

After filtering, click **“Build Listing Choices”**.  
Then:
- Select your **reference property**.
- Choose a **target country**.
- Set **K**, the number of similar listings to return.


In [0]:
out_choose = widgets.Output()

w_ref = widgets.Dropdown(
    options=[],
    description="Listing",
    layout=widgets.Layout(width="900px")
)

w_target_country = widgets.Dropdown(
    options=countries,
    description="Target Country",
    layout=widgets.Layout(width="180px")
)

N_CANDIDATES_FIXED = 50
# w_n_candidates = widgets.IntText(
#     value=N_CANDIDATES_FIXED,
#     description="Candidates",
#     disabled=True,
#     layout=widgets.Layout(width="220px")
# )

w_k_show = widgets.Dropdown(
    options=[5, 10, 15, 20],
    value=10,
    description="Top-K",
    layout=widgets.Layout(width="180px")
)

btn_build_choices = widgets.Button(
    description="Build listing choices",
    button_style="info"
)

def build_listing_options():
    rows = STATE.get("filtered_rows") or []
    opts = []
    for r in rows:
        d = r.asDict()
        pid = d.get("property_id")
        title = d.get("listing_title") or ""
        city = d.get("addr_name") or ""
        cc = d.get("addr_cc") or ""
        price = d.get("price_per_night")
        rating = d.get("ratings")
        label = f"{title} — {city} ({cc}) | €{price} | ⭐ {rating}"
        opts.append((label, str(pid)))
    return opts

def on_build(_):
    with out_choose:
        clear_output()
        opts = build_listing_options()
        if not opts:
            print("No filtered rows yet. Run the filter block first.")
            return
        w_ref.options = opts
        w_ref.value = opts[0][1]
        print("Listing options ready.")

btn_build_choices.on_click(on_build)

display(
    widgets.VBox([
        widgets.HBox([btn_build_choices, ]),
        w_ref,
        w_target_country,
        widgets.HBox([w_k_show]),
        out_choose
    ])
)


## Step 3: Set Your Preferences and Run the Recommendation

Adjust the preference sliders to reflect what matters most to you.  
When you are ready, click **“Run Recommend”** and wait for the results to appear.


In [0]:
import ipywidgets as widgets

w_price_w = widgets.IntSlider(
    value=25, min=0, max=100, step=5,
    description="How important is price?",
    continuous_update=False,
    layout=widgets.Layout(width="520px"),
    style={"description_width": "230px"}
)

w_property_w = widgets.IntSlider(
    value=25, min=0, max=100, step=5,
    description="How important is property quality?",
    continuous_update=False,
    layout=widgets.Layout(width="520px"),
    style={"description_width": "230px"}
)

w_host_w = widgets.IntSlider(
    value=25, min=0, max=100, step=5,
    description="How important is host quality?",
    continuous_update=False,
    layout=widgets.Layout(width="520px"),
    style={"description_width": "230px"}
)

# ---- Optional context questions ----
w_temp_w = widgets.IntSlider(
    value=0, min=0, max=100, step=5,
    description="How important is weather match?",
    continuous_update=False,
    layout=widgets.Layout(width="520px"),
    style={"description_width": "230px"}
)

w_budget_w = widgets.IntSlider(
    value=0, min=0, max=100, step=5,
    description="How important is staying within a budget level?",
    continuous_update=False,
    layout=widgets.Layout(width="520px"),
    style={"description_width": "230px"}
)

w_temp_pref = widgets.FloatText(
    value=22.0,
    description="Preferred temperature (°C)",
    layout=widgets.Layout(width="260px"),
    style={"description_width": "180px"}
)

w_month = widgets.IntSlider(
    value=7, min=1, max=12, step=1,
    description="Travel month",
    continuous_update=False,
    layout=widgets.Layout(width="420px"),
    style={"description_width": "120px"}
)

w_budget_pref = widgets.Dropdown(
    options=["", "low", "mid", "high"],
    value="",
    description="Budget level",
    layout=widgets.Layout(width="260px"),
    style={"description_width": "120px"}
)

def pretty_env_name(col: str) -> str:
    x = col.replace("env_", "").replace("_norm", "")
    return x.replace("_", " ").title()

env_sliders = {}
for c in ENV_COLS:
    env_sliders[c] = widgets.IntSlider(
        value=0, min=0, max=100, step=5,
        description=f"How important is {pretty_env_name(c)}?",
        continuous_update=False,
        layout=widgets.Layout(width="720px"),
        style={"description_width": "300px"}
    )

env_box = widgets.VBox(list(env_sliders.values())) if env_sliders else widgets.HTML("<i>No env_* columns found.</i>")

display(widgets.VBox([
    widgets.HTML("<h3>Step 3: Set Your Preferences</h3>"),
    widgets.HTML("<p class='muted'>Move the sliders to reflect what matters most to you.</p>"),
    widgets.HTML("<h4>Main preferences</h4>"),
    widgets.VBox([w_price_w, w_property_w, w_host_w]),
    widgets.HTML("<h4>Optional preferences</h4>"),
    widgets.VBox([w_temp_w, w_budget_w]),
    widgets.HBox([w_temp_pref, w_month, w_budget_pref]),
    widgets.HTML("<h4>Neighborhood vibe</h4>"),
    env_box
]))


In [0]:
# ----------------------------
# STEP 3 — Run recommend (Spark)
# ----------------------------
btn_run = widgets.Button(description="Run Recommend", button_style="success")
out_run = widgets.Output()

def on_run(_):
    with out_run:
        clear_output(wait=True)

        target_id = w_ref.value
        target_country = w_target_country.value
        n_candidates = N_CANDIDATES_FIXED
        k_show = int(w_k_show.value)

        if not target_id:
            print("Pick a reference listing first (Step 2).")
            return
        if not target_country:
            print("Pick a target country (Step 2).")
            return

        # env weights from sliders (expects env_* or env_*_norm keys)
        env_weights = {col: int(slider.value) for col, slider in env_sliders.items() if int(slider.value) != 0}

        params = dict(
            w_price=float(w_price_w.value),
            w_property=float(w_property_w.value),
            w_host=float(w_host_w.value),
            w_temp=float(w_temp_w.value),
            w_budget=float(w_budget_w.value),
            temp_pref=float(w_temp_pref.value) if w_temp_pref.value not in (None, "") else None,
            travel_month=int(w_month.value) if w_month.value not in (None, "") else None,
            budget_pref=(w_budget_pref.value or None),
            env_weights=env_weights
        )

        print("Running recommend...")
        # print(" Reference listing:", target_id)
        # print(" Target country:", target_country)
        # print(" Candidates:", n_candidates, "| Top-K:", k_show)

        t0 = time.perf_counter()

        # Retrieve candidates (Spark)
        cand_df = retrieve(
            target_id=target_id,
            country=target_country,
            df=df_emb,
            lsh_model=lsh_model,
            n=n_candidates
        )
        if cand_df is None:
            print("retrieve() returned None (target missing embedding?)")
            return

        cand_df = cand_df.filter(F.col("property_id") != F.lit(target_id))

        # Join for display + ranking columns
        cand_cols = set(cand_df.columns)
        df_all_to_join = df_all.select(*[c for c in df_all.columns if (c == "property_id") or (c not in cand_cols)])
        cand_df = cand_df.join(df_all_to_join, on="property_id", how="inner")

        # Rank (Spark)
        ranked = order(
            df=cand_df,
            k=k_show,
            price_w=params["w_price"],
            property_w=params["w_property"],
            host_w=params["w_host"],
            env_weights=params["env_weights"],
            temp_pref=params["temp_pref"],
            temp_w=params["w_temp"],
            travel_month=params["travel_month"],
            budget_pref=params["budget_pref"],
            budget_w=params["w_budget"],
            normalize_all_weights=True,
            score_col="final_score"
        )

        # Final ordering (optional tie-break)
        final_topk = ranked.orderBy(F.desc("final_score"), F.asc_nulls_last("l2_dist")).limit(k_show)
        STATE["final_topk"] = final_topk

        t1 = time.perf_counter()
        print(f"Done in {(t1 - t0):.2f}s")

        # DISPLAY — NO ID
        show_spark_df(
            final_topk,
            n=k_show,
            title="Top-K Recommendations",
            cols=["listing_title","addr_cc","addr_name","room_type_text","price_per_night","ratings","final_url"],
            rename_map=DISPLAY_COL_RENAME,
            output_widget=out_run
        )

btn_run.on_click(on_run)

ipy_display(widgets.VBox([btn_run, out_run]))
